In [1]:
#!pip install pyarrow

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split

In [3]:
#CONSTANTS
exercises_dict = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat' : 5,
    'Stretch' : 6
}

In [4]:
dataset = pd.read_parquet('dataset.parquet', engine='pyarrow')

In [5]:
data = dataset.values
indices = dataset.index
labels = dataset.columns

x_train = data[:,4:]
y_train = [exercises_dict[e] for e in data[:,1]]

In [6]:
#example of a row with labels
print(np.stack((labels.to_numpy(),data[0]), axis = 1))

[['Participant' 'P04']
 ['Exercise' 'Abduction']
 ['Set' 'Correct']
 ['Camera' 'Frontal_Top']
 ['time(s)' 0.0]
 ['left_ankle_x' 0.009327623061835766]
 ['left_ankle_y' 0.7793046236038208]
 ['left_ankle_z' 0.18788036704063416]
 ['left_ear_x' 0.02065831422805786]
 ['left_ear_y' -0.6934192776679993]
 ['left_ear_z' 0.023042714223265648]
 ['left_elbow_x' -0.023044275119900703]
 ['left_elbow_y' -0.472626656293869]
 ['left_elbow_z' 0.41818660497665405]
 ['left_eye_center_x' 0.10780128091573715]
 ['left_eye_center_y' -0.690135657787323]
 ['left_eye_center_z' -0.044566456228494644]
 ['left_eye_inner_x' 0.10743892192840576]
 ['left_eye_inner_y' -0.6897681951522827]
 ['left_eye_inner_z' -0.045617058873176575]
 ['left_eye_outer_x' 0.10706208646297455]
 ['left_eye_outer_y' -0.6908190250396729]
 ['left_eye_outer_z' -0.04443546012043953]
 ['left_foot_x' 0.1335529088973999]
 ['left_foot_y' 0.8766452670097351]
 ['left_foot_z' 0.13864915072917938]
 ['left_heel_x' 0.019140124320983887]
 ['left_heel_y' 0.8

In [8]:
#Splits the dataset in a 4-channel data set (think of an RGB Image) coresponding to the 4 different cameras

lengths = np.append(np.where(indices.array[:len(indices)] == 0)[0], [len(indices)])
ft_x, fl_x, st_x, sl_x = [], [], [], []
mod4 = 0
gap = 0

for idx, l in enumerate(lengths):
    if idx < len(lengths)-1:
        end = lengths[idx + 1]
        
        if mod4 == 0:
            gap = end - l

        segment = x_train[l:l + gap]
        
        if mod4 == 0:
            ft_x.append(segment)
        elif mod4 == 1:
            fl_x.append(segment)

        elif mod4 == 2:
            st_x.append(segment)

        elif mod4 == 3:
            sl_x.append(segment)

        
        mod4 = (mod4 + 1) % 4

print("STACKING:")
# Concatenate the lists into arrays
ft_x = np.vstack(ft_x) 
fl_x = np.vstack(fl_x)
st_x = np.vstack(st_x) 
sl_x = np.vstack(sl_x)

x_train_reshape = np.array([ft_x, fl_x, st_x, sl_x])
print(x_train_reshape.shape)

STACKING:
(4, 552572, 100)
